In [1]:
import numpy as np
import pandas as pd
 
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import xgboost as xgb
from lightgbm import LGBMRegressor
import math
%matplotlib inline


C:\Users\Alex\Anaconda3\envs\python36\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\Alex\Anaconda3\envs\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
rolling_mask_two = [-i for i in range(1,2)]+[i for i in range(1,2)]
rolling_mask_four = [-i for i in range(1,3)]+[i for i in range(1,3)]
rolling_mask_six = [-i for i in range(1,4)]+[i for i in range(1,4)]
rolling_mask_eight = [-i for i in range(1,5)]+[i for i in range(1,5)]
rolling_mask_ten = [-i for i in range(1,6)]+[i for i in range(1,6)]

In [3]:
# 数据预处理
# def drop_all_outlier(df):
#     df.drop_duplicates(df.columns.drop('ID'), keep='first', inplace=True)
#     df.drop(df[(df.V_A > 800) | (df.V_A < 500)].index,inplace=True)
#     df.drop(df[(df.V_B > 800) | (df.V_B < 500)].index,inplace=True)
#     df.drop(df[(df.V_C > 800) | (df.V_C < 500)].index,inplace=True)
#     df.drop(df[(df.env_t > 30) | (df.env_t < -30)].index,inplace=True)
# #     df.drop(df[(df.转换效率A > 100)].index,inplace=True)
# #     df.drop(df[(df.转换效率B > 100)].index,inplace=True)
# #     df.drop(df[(df.转换效率C > 100)].index,inplace=True)
# #     df.drop(df[(df.wind_direction > 360)].index,inplace=True)
#     df.drop(df[(df.wind_speed > 20)].index,inplace=True)
#     return df

# 生成数据
def generate_train_data(train_data, test_data, poly=False, select=False):
    y = train_data['y']
    X = train_data.drop(['y','ID','is_train'], axis=1)
    sub_data = test_data.drop(['y','ID','is_train'], axis=1)
    
    polynm = None
    if poly:
        from sklearn.preprocessing import PolynomialFeatures
        polynm = PolynomialFeatures(degree=2, interaction_only=True)
        X = polynm.fit_transform(X)
        sub_data = polynm.transform(sub_data)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
    
    sm = None
    if select:
        from sklearn.feature_selection import SelectFromModel
        sm = SelectFromModel(GradientBoostingRegressor(random_state=2))
        X_train = sm.fit_transform(X_train, y_train)
        X_test = sm.transform(X_test)
        sub_data = sm.transform(sub_data)
        
    return X_train, X_test, y_train, y_test, sub_data, sm, polynm

def cal_score(mse):
    if isinstance(mse, float):
        return 1 / (1 + math.sqrt(mse))
    else:
        return np.divide(1, 1 + np.sqrt(mse))
#  定义交叉验证函数  
def cross_validation_test(models, train_X_data, train_y_data, cv=5):
    model_name, mse_avg, score_avg = [], [], []
    for i, model in enumerate(models):
        print(i + 1,'- Model:', str(model).split('(')[0])
        model_name.append(str(i + 1) + '.' + str(model).split('(')[0])
        nmse = cross_val_score(model, train_X_data[i], train_y_data[i], cv=cv, scoring='neg_mean_squared_error')
        avg_mse = np.average(-nmse)
        scores = cal_score(-nmse)
        avg_score = np.average(scores)
        mse_avg.append(avg_mse)
        score_avg.append(avg_score)
        print('MSE:', -nmse)
        print('Score:', scores)
        print('Average XGB - MSE:', avg_mse, ' - Score:', avg_score, '\n')
    res = pd.DataFrame()
    res['Model'] = model_name
    res['Avg MSE'] = mse_avg
    res['Avg Score'] = score_avg
    return res

# def add_newid(df):
#     ID = df["ID"]
#     df["new_id"]=(np.mod(ID,205))
#     return df
# def add_avg(df):
#     array = np.array(df["P_avg"])
#     newarray=[]
#     num = 0
#     for i in np.arange(len(array)):
#         for j in np.arange(10):
#             if i<10:
#                 num = (array[j-1]+array[j-2]+array[j-3])/3
#             if i>=10:
#                 num = (array[i-1]+array[i-2]+array[i-3]+array[i-5]+array[i-6]+array[i-7]+array[i-8]+array[i-9])/9
#         newarray.append(num)
#     df["old_SoCalledSF_P_avg"] = newarray
#     return df

# 原始数据导入

In [4]:
train = pd.read_csv('../data/public_raw.train.csv')
test = pd.read_csv('../data/public_raw.test.csv')

train['is_train']=1
test['is_train']=0

df = pd.concat([train, test],sort=False)

rep_cols = {'ID':'ID', 
 '板温':'board_t', 
 '现场温度':'env_t', 
 '光照强度':'light_strength', 
 '转换效率':'efficiency', 
 '转换效率A':'efficiency_A', 
 '转换效率B':'efficiency_B', 
 '转换效率C':'efficiency_C', 
 '电压A':'V_A',
 '电压B':'V_B', 
 '电压C':'V_C', 
 '电流A':'I_A', 
 '电流B':'I_B', 
 '电流C':'I_C', 
 '功率A':'P_A', 
 '功率B':'P_B', 
 '功率C':'P_C', 
 '平均功率':'P_avg', 
 '风速':'wind_speed',
 '风向':'wind_direction', 
 '发电量':'y'
}

df.rename(index=str, columns=rep_cols, inplace=True)

df.sort_values(by=['ID'],ascending=True, inplace=True)

df.reset_index(drop=True, inplace=True)

# train_data.rename(index=str, columns=rep_cols, inplace=True)
# test_data.rename(index=str, columns=rep_cols, inplace=True)

# 数据清洗

In [5]:
#清洗原因一：人工发现训练集和测试集合中均有很多样本，测量值均为零，发电量为0.379993053。
#于是有两个方向，方向一：将此类样本是为异常样本，认为不存在学习价值，使用最粗暴的方式，从训练集中剔除这些样本。在测试集中人工赋值0.379993053。
#方向二：认为这些样本不是异常样本，存在学习价值。同时将测试集合中此类样本增加到训练样本中，让分布更逼近真实分布。

#首先尝试方向一：
# special_missing_ID = test_data[test_data[(test_data == 0) | (test_data == 0.)].count(axis=1) > 13]['ID']

In [6]:
#清洗原因二：电压、电流、温度、风速这些连续测量值中存在明显的异常值，违背物理常识
#如何定义异常？
#情况一：在一个合理的时间段内，一个时刻的测量值与其他时刻测量值的均值的差异很大，差异如何刻画？

In [8]:
#原始路线
all_data = df.copy()
bad_feature = ['ID','P_A', 'P_B', 'P_C', 'P_avg', 'env_t', 'V_A', 'V_B', 'V_C', 'I_B', 'I_C', 'efficiency', 'efficiency_A', 'efficiency_B', 'efficiency_C']
bad_index1 = all_data[bad _feature][
    (all_data[bad_feature] > all_data[bad_feature].mean() + 2 * all_data[bad_feature].std()) | 
    (all_data[bad_feature] < all_data[bad_feature].mean() - 2 * all_data[bad_feature].std())
].dropna(how='all').index
bad_index2 = all_data[
    ((all_data['V_A']<500)&(all_data['V_A']!=0))|
    ((all_data['V_B']<500)&(all_data['V_B']!=0))|
    ((all_data['V_C']<500)&(all_data['V_C']!=0))].index
bad_index = pd.Int64Index(list(bad_index1)+list(bad_index2))
# bad_index = all_data[bad_feature][
#     (all_data[bad_feature] > all_data[bad_feature].mean() + 2 * all_data[bad_feature].std()) | 
#     (all_data[bad_feature] < all_data[bad_feature].mean() - 2 * all_data[bad_feature].std())
# ].dropna(how='all').index



bad_data = all_data.loc[bad_index].sort_values(by='ID', ascending=True)





# 上下记录均值替代异常值
for idx, line in bad_data.iterrows():
    ID = line['ID']
    col_index = line[bad_feature][ 
        (line[bad_feature] > all_data[bad_feature].mean() + 3 * all_data[bad_feature].std())| 
        (line[bad_feature] < all_data[bad_feature].mean() - 3 * all_data[bad_feature].std())
    ].index
    index = all_data[all_data['ID'] == ID].index
    
    before_offset = 1
    while (idx + before_offset)in bad_index:
        before_offset += 1

    after_offset = 1
    while (idx + after_offset) in bad_index:
        after_offset += 1
    
    replace_value = (all_data.loc[index - before_offset, col_index].values + all_data.loc[index + after_offset, col_index].values) / 2
    all_data.loc[index, col_index] = replace_value[0]

SyntaxError: invalid syntax (<ipython-input-8-bb2a10d1fd26>, line 4)

In [ ]:
#前二后二
next_one = []
prev_one = []
next_id = []
prev_id = []

second_next_one = []
second_prev_one = []

df_len = df.shape[0]

i_y =df.columns.get_loc("y")

def get_prev_nn_index(cur_i):
    prev_i = cur_i-1
    while(prev_i>=0 and pd.isnull(df.iat[prev_i,i_y])):
        prev_i-=1
    return prev_i

def get_next_nn_index(cur_i):
    prev_i = cur_i+1
    while(prev_i<df_len and pd.isnull(df.iat[prev_i,i_y])):
        prev_i+=1
    return prev_i

for i in range(df_len):
    f_pre_i=get_prev_nn_index(i)
    if(f_pre_i)<0:
        prev_one.append(np.nan)
        prev_id.append(0)
    else:
        prev_one.append(df.iat[f_pre_i,i_y])
        prev_id.append(f_pre_i)
        
    s_pre_i=get_prev_nn_index(f_pre_i)
    if (s_pre_i)<0:
        second_prev_one.append(np.nan)
    else:
        second_prev_one.append(df.iat[s_pre_i,i_y])
    
    f_next_i=get_next_nn_index(i)
    if(f_next_i<df_len):
        next_one.append(df.iat[f_next_i,i_y])
        next_id.append(f_next_i)
    else:
        next_one.append(np.nan)
        next_id.append(df_len)
    
    s_next_i=get_next_nn_index(f_next_i)
    if(s_next_i<df_len):
        second_next_one.append(df.iat[s_next_i,i_y])
    else:
        second_next_one.append(np.nan)
        

df['next_value'] = next_one
df['prev_value'] = prev_one
df['avg_value'] = np.nanmean([df['next_value'], df['prev_value']],axis=0)

df.drop(['next_value','prev_value'],1,inplace=True)

In [ ]:
#拆分数据

train_data = all_data[all_data['is_train']==1]
test_data = all_data[all_data['is_train']==0]
len(train_data), len(test_data)

train_data = train_data.drop_duplicates(train_data.columns.drop('ID'), keep='first')

In [ ]:
df_result = pd.DataFrame()
df_result['ID'] = list(test_data['ID'])
special_missing_ID = test_data[test_data[(test_data == 0) | (test_data == 0.)].count(axis=1) > 13]['ID']



In [ ]:


X_train, X_test, y_train, y_test, sub_data, sm, polynm = generate_train_data(train_data, test_data, poly=True, select=True)
# clean_X_train, clean_X_test, clean_y_train, clean_y_test, clean_sub_data, sm1, polynm1 = generate_train_data(cleaned_train_data, cleaned_test_data, poly=True, select=True)

all_X_train = np.concatenate([X_train, X_test])
all_y_train = np.concatenate([y_train, y_test])

# 试验休了，时长为前后各4个时刻，偏差率为1.6，以此作为初值

In [9]:
#计算偏差率的辅助列
for c in ['I_A','I_B','I_C','V_A','V_B','V_C']:
    df[c+'_avg_sequence'] = np.nanmean([df[c].shift(i) for i in rolling_mask_eight],axis=0)
    df[c+'_exception_ratio'] = np.abs(df[c]-df[c+'_avg_sequence'])/df[c+'_avg_sequence']

In [16]:
#标记包含异常值的记录
df['is_abnormal']=0

for c in ['I_A','I_B','I_C','V_A','V_B','V_C']:
    df.loc[df[c+'_exception_ratio'] > 10 , 'is_abnormal'] = 1



In [14]:
# for c in ['I_A','I_B','I_C','V_A','V_B','V_C']:
    
#     df.loc[df[c+'_exception_ratio'] > 1.6 , 'is_abnormal'] = 1

In [17]:
#训练集中异常记录30条
df[df['is_train']==1][df['is_abnormal']==1][['ID','board_t','light_strength','I_A','I_A_exception_ratio','I_B','I_C','V_A','V_B','V_C','P_avg','y']]

C:\Users\Alex\Anaconda3\envs\python36\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,ID,board_t,light_strength,I_A,I_A_exception_ratio,I_B,I_C,V_A,V_B,V_C,P_avg,y
14,22,-10.77,100,7.19,1.520596,645.39,2.75,65382,7,107,158302.85,5.440741
1065,1173,-5.83,233,7.04,0.277967,649.75,2.75,65408,22,250,158484.77,7.753474
1178,1286,0.44,100,7.18,1.397329,7.16,7.18,310,307,65438,158089.59,4.694385
1237,1362,-3.70,233,7.00,0.316718,652.02,2.79,65386,23,244,157793.07,7.806384
1326,1451,2.56,262,6.84,0.258220,6.82,3.12,560,65454,27,150103.64,7.996870
1373,1519,0.25,9,7.04,6.673025,7.10,639.04,77,65387,14,157912.11,0.839478
1520,1666,0.00,112,6.87,1.506156,6.80,6.81,293,255,65460,149843.17,4.521973
1722,1894,0.24,30,7.10,8.726027,7.13,635.60,74,65350,22,160151.37,0.780917
2214,2414,0.20,115,7.02,1.529730,7.00,6.99,292,296,65470,153919.05,4.299538
2567,2797,0.92,94,6.86,1.380911,6.86,3.60,305,65512,17,150521.94,4.221675


In [ ]:
#训练集中异常记录34条
df[df['is_train']==0][df['is_abnormal']==1][['ID','board_t','light_strength','I_A','I_B','I_C','V_A','V_B','V_C','P_avg','y']]

In [ ]:
df.shape

In [ ]:
#电学测量值全为0的测试集记录,ID存在special_missing_ID
special_missing_ID=df[df[(df == 0) | (df == 0.)].count(axis=1) > 13][df['is_train']==0]['ID']
special_missing_ID.shape

In [ ]:
#异常值由滚动平均值替代（前后各4个点的平均值）
static=1
for idx, line in df.iterrows():
    for c in ['I_A','I_B','I_C','V_A','V_B','V_C']:
        if line[c+'_exception_ratio']>1.6:
            print(str(line[c]) + ' is abnormal as value of ' + c)
            print('Mark for abnormal records: ' + str(line['is_abnormal']))
            line.loc[c] = line[c+'_avg_sequence']
            print('Has been replaced by '+str(line[c+'_avg_sequence'])) 
            static += 1
            print(static)

In [ ]:
df.shape

In [ ]:
#去除辅助行
def drop_cols(df):
    col_list=[] 
    for c in ['I_A','I_B','I_C','V_A','V_B','V_C']:
        col_list.append(c+'_exception_ratio')
        col_list.append(c+'_avg_sequence')
    df.drop(col_list,axis=1,inplace=True)
    return df

In [ ]:
df = drop_cols(df)
df.drop('is_abnormal',axis=1,inplace=True)
df.head()

In [ ]:
df.drop_duplicates(df.columns.drop('ID'), keep='first', inplace=True)

In [ ]:
df.shape

In [ ]:
#二次项特征

# 生成数据
def generate_train_data(train_data, test_data, poly=False, select=False):
    y = train_data['y']
    X = train_data.drop(['y','ID','is_train'], axis=1)
    sub_data = test_data.drop(['y','ID','is_train'], axis=1)
    
    polynm = None
    if poly:
        from sklearn.preprocessing import PolynomialFeatures
        polynm = PolynomialFeatures(degree=2, interaction_only=True)
        X = polynm.fit_transform(X)
        sub_data = polynm.transform(sub_data)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
    
    sm = None
    if select:
        from sklearn.feature_selection import SelectFromModel
        sm = SelectFromModel(GradientBoostingRegressor(random_state=2))
        X_train = sm.fit_transform(X_train, y_train)
        X_test = sm.transform(X_test)
        sub_data = sm.transform(sub_data)
        
    return X_train, X_test, y_train, y_test, sub_data, sm, polynm

#train和test拆分
train_data = df[df['is_train']==1]
test_data = df[df['is_train']==0]

# cleaned_train_data = train_data[train_data['is_abnormal']==0]
# cleaned_test_data = test_data[test_data['is_abnormal']==0]


# train_data = add_avg(train_data)
# test_data = add_avg(test_data)
# cleaned_train_data = add_avg(cleaned_train_data)
# cleaned_test_data = add_avg(cleaned_test_data)


X_train, X_test, y_train, y_test, sub_data, sm, polynm = generate_train_data(train_data, test_data, poly=True, select=True)
# clean_X_train, clean_X_test, clean_y_train, clean_y_test, clean_sub_data, sm1, polynm1 = generate_train_data(cleaned_train_data, cleaned_test_data, poly=True, select=True)

# clean_X = np.concatenate([clean_X_train, clean_X_test])
# clean_y = np.concatenate([clean_y_train, clean_y_test])

all_X_train = np.concatenate([X_train, X_test])
all_y_train = np.concatenate([y_train, y_test])

In [ ]:
# def drop_all_outlier(df):
#     for c in ['I_A','I_B','I_C','V_A','V_B','V_C']:
#         df.drop(df[(df[c+'_exception_ratio'] > 1.6)].index,inplace=True)
#     df.drop(df[(df.env_t > 30) | (df.env_t < -30)].index,inplace=True)
#     df.drop(df[(df.wind_speed > 20)].index,inplace=True)

#     return df

In [ ]:
# #train和test拆分
# train_data = df[df['is_train']==1]
# test_data = df[df['is_train']==0]

In [ ]:
# #准备测试结果
# df_result = pd.DataFrame()
# df_result['ID'] = list(test_data['ID'])


In [ ]:
# special_missing_ID = test_data[test_data[(test_data == 0) | (test_data == 0.)].count(axis=1) > 13]['ID']

In [ ]:
# cleaned_train_data = train_data.copy()
# cleaned_train_data = drop_all_outlier(cleaned_train_data)

# cleaned_sub_data = test_data.copy()
# cleaned_sub_data = drop_all_outlier(cleaned_sub_data)
# cleaned_sub_data_ID = cleaned_sub_data['ID']

In [ ]:
all_data  = pd.concat([train_data, test_data], axis=0).sort_values(by='ID').reset_index().drop(['index'], axis=1)
bad_feature = ['ID','P_A', 'P_B', 'P_C', 'P_avg', 'env_t', 'V_A', 'V_B', 'V_C', 'I_B', 'I_C', 'efficiency', 'efficiency_A', 'efficiency_B', 'efficiency_C']
bad_index1 = all_data[bad_feature][
    (all_data[bad_feature] > all_data[bad_feature].mean() + 2 * all_data[bad_feature].std()) | 
    (all_data[bad_feature] < all_data[bad_feature].mean() - 2 * all_data[bad_feature].std())
].dropna(how='all').index
bad_index2 = all_data[
    ((all_data['V_A']<500)&(all_data['V_A']!=0))|
    ((all_data['V_B']<500)&(all_data['V_B']!=0))|
    ((all_data['V_C']<500)&(all_data['V_C']!=0))].index
bad_index = pd.Int64Index(list(bad_index1)+list(bad_index2))
# all_data.loc[np.concatenate([bad_index -1,bad_index,bad_index+1])].sort_values(by='ID', ascending=True)


nn_bad_data = all_data.loc[np.concatenate([bad_index - 1, bad_index, bad_index + 1])].sort_values(by='ID', ascending=True).drop_duplicates()
bad_data = all_data.loc[bad_index].sort_values(by='ID', ascending=True)

In [ ]:
# 上下记录均值替代异常值
for idx, line in bad_data.iterrows():
    ID = line['ID']
    col_index = line[bad_feature][ 
        (line[bad_feature] > all_data[bad_feature].mean() + 3 * all_data[bad_feature].std())| 
        (line[bad_feature] < all_data[bad_feature].mean() - 3 * all_data[bad_feature].std())
    ].index
    index = all_data[all_data['ID'] == ID].index
    
    before_offset = 1
    while (idx + before_offset)in bad_index:
        before_offset += 1

    after_offset = 1
    while (idx + after_offset) in bad_index:
        after_offset += 1
    
    replace_value = (all_data.loc[index - before_offset, col_index].values + all_data.loc[index + after_offset, col_index].values) / 2
    all_data.loc[index, col_index] = replace_value[0]

In [ ]:
# #异常值被前后各4个点的平均值替代
# for idx, line in df.iterrows():
#     for c in ['I_A','I_B','I_C','V_A','V_B','V_C']:
#         if line[c+'_exception_ratio']>1.6:
#             line.loc[c] = line[c+'_avg_sequence']
#             print(str(line[c]) + 'is replaced by '+str(line[c+'_avg_sequence'])) 

In [ ]:
# #拆分数据
# train_data = df[df['is_train']==1].reset_index().drop(['index'], axis=1)
# test_data = df[df['is_train']==0].drop(['y'], axis=1).reset_index().drop(['index'], axis=1)

In [ ]:
# #拆分数据
# train_data = all_data.drop(all_data[all_data['ID'].isin(df_result['ID'])].index).reset_index().drop(['index'], axis=1)
# test_data = all_data[all_data['ID'].isin(df_result['ID'])].drop(['y'], axis=1).reset_index().drop(['index'], axis=1)
# len(train_data), len(test_data)
# # 去除重复值
# train_data = train_data.drop_duplicates(train_data.columns.drop('ID'), keep='first')

In [ ]:
# train_data = add_avg(train_data)
# test_data = add_avg(test_data)
# cleaned_train_data = add_avg(cleaned_train_data)
# cleaned_sub_data = add_avg(cleaned_sub_data)

In [ ]:
# cleaned_sub_data = cleaned_sub_data.drop(['y'], axis=1).reset_index().drop(['index'], axis=1)

In [ ]:
# cleaned_train_data.head()

In [ ]:
# def drop_cols(df):
#     col_list=[] 
#     for c in ['I_A','I_B','I_C','V_A','V_B','V_C']:
#         col_list.append(c+'_exception_ratio')
#         col_list.append(c+'_avg_sequence')
#     df.drop(col_list,axis=1,inplace=True)
#     return df


In [ ]:
# train_data=drop_cols(train_data)
# test_data = drop_cols(test_data)
# cleaned_train_data = drop_cols(cleaned_train_data)
# cleaned_sub_data = drop_cols(cleaned_sub_data)

In [ ]:
# X_train, X_test, y_train, y_test, sub_data, sm, polynm = generate_train_data(train_data, test_data, poly=True, select=True)

# clean_X_train, clean_X_test, clean_y_train, clean_y_test, clean_sub_data, _, _ = generate_train_data(cleaned_train_data, cleaned_sub_data, poly=False, select=False)

# clean_X = np.concatenate([clean_X_train, clean_X_test])
# clean_y = np.concatenate([clean_y_train, clean_y_test])
# clean_X = polynm.transform(clean_X)
# clean_X = sm.transform(clean_X)

# clean_sub_data = polynm.transform(clean_sub_data)
# clean_sub_data = sm.transform(clean_sub_data)

In [ ]:
# xgbt1 = xgb.XGBRegressor(n_estimators=950, max_depth=3, max_features='sqrt', random_state=321, n_jobs=8)
# xgbt2 = xgb.XGBRegressor(n_estimators=1000, max_depth=3, max_features='sqrt', random_state=456, n_jobs=8)
xgbt3 = xgb.XGBRegressor(n_estimators=1100, max_depth=3, max_features='sqrt', random_state=789, n_jobs=8)
# n_estimators=1000  max_depth=5  'sqrt'  GradientBoostingRegressor 最佳参数 ,learning_rate=0.08
# gbdt1 = GradientBoostingRegressor(n_estimators=800, max_depth=4, max_features='log2', random_state=123,learning_rate=0.08)
# gbdt2 = GradientBoostingRegressor(n_estimators=900, max_depth=4, max_features='log2', random_state=456,learning_rate=0.08)
gbdt3 = GradientBoostingRegressor(n_estimators=1000, max_depth=5, max_features='log2', random_state=789,learning_rate=0.08)
# n_estimators=700, max_features='auto', random_state=2, n_jobs=8,max_depth=10
# forest1 = RandomForestRegressor(n_estimators=800, max_features='sqrt', random_state=7, n_jobs=8)
# forest2 = RandomForestRegressor(n_estimators=900, max_features='log2', random_state=9, n_jobs=8)
forest3 = RandomForestRegressor(n_estimators=900, max_features='sqrt', random_state=11, n_jobs=8) 

# lgb1 = LGBMRegressor(n_estimators=900, max_depth=5, random_state=5, n_jobs=8) 
# lgb2 = LGBMRegressor(n_estimators=850, max_depth=4, random_state=7, n_jobs=8)
lgb3 = LGBMRegressor(n_estimators=720, max_depth=4, random_state=9, n_jobs=8)

# xgbt1 = xgb.XGBRegressor(n_estimators=950, max_depth=3, max_features='sqrt', random_state=2, n_jobs=8)
# xgbt2 = xgb.XGBRegressor(n_estimators=1000, max_depth=3, max_features='sqrt', random_state=3, n_jobs=8)
# xgbt3 = xgb.XGBRegressor(n_estimators=1100, max_depth=3, max_features='sqrt', random_state=4, n_jobs=8)

# gbdt1 = GradientBoostingRegressor(n_estimators=500, max_depth=3, max_features='sqrt', random_state=2)
# gbdt2 = GradientBoostingRegressor(n_estimators=400, max_depth=3, max_features='sqrt', random_state=3)
# gbdt3 = GradientBoostingRegressor(n_estimators=500, max_depth=4, max_features='log2', random_state=4)

# forest1 = RandomForestRegressor(n_estimators=300, max_features='sqrt', random_state=2, n_jobs=8)
# forest2 = RandomForestRegressor(n_estimators=300, max_features='log2', random_state=3, n_jobs=8)
# forest3 = RandomForestRegressor(n_estimators=600, max_features='sqrt', random_state=4, n_jobs=8) 

# lgb1 = LGBMRegressor(n_estimators=900, max_depth=5, random_state=2, n_jobs=8) 
# lgb2 = LGBMRegressor(n_estimators=850, max_depth=4, random_state=3, n_jobs=8)
# lgb3 = LGBMRegressor(n_estimators=720, max_depth=4, random_state=4, n_jobs=8)

# cross_validation_test(
#     models=[    
#         xgbt1, xgbt2, xgbt3,
#         gbdt1, gbdt2, gbdt3,
#         forest1, forest2, forest3,
#         lgb1, lgb2, lgb3
#     ],
#     train_X_data=[
#         all_X_train, all_X_train, all_X_train, all_X_train,
#         all_X_train, all_X_train, all_X_train, all_X_train,
#         all_X_train, all_X_train, all_X_train, all_X_train
#     ],
#     train_y_data=[
#         all_y_train, all_y_train, all_y_train, all_y_train,
#         all_y_train, all_y_train, all_y_train, all_y_train,
#         all_y_train, all_y_train, all_y_train, all_y_train
#     ]
# )

In [ ]:
regrs = [
    xgbt1, gbdt1, forest1, lgb1,
    xgbt2, gbdt2, forest2, lgb2,
    xgbt3, gbdt3, forest3, lgb3
]

In [ ]:
regrs = [
    xgbt3, gbdt3, forest3, lgb3
]

In [ ]:
class Stacker(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models
    
    # X: 原始训练集, y: 原始训练集真实值, predict_data: 原始待预测数据
    def fit_predict(self, X, y, predict_data):
        X = np.array(X)
        y = np.array(y)
        T = np.array(predict_data)

        folds = list(KFold(n_splits=self.n_splits, shuffle=False, random_state=2018).split(X, y))
        
        # 以基学习器预测结果为特征的 stacker的训练数据 与 stacker预测数据
        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_predict = np.zeros((T.shape[0], len(self.base_models)))
        
        for i, regr in enumerate(self.base_models):
            print(i + 1, 'Base model:', str(regr).split('(')[0])
            S_predict_i = np.zeros((T.shape[0], self.n_splits))
            
            for j, (train_idx, test_idx) in enumerate(folds):
                # 将X分为训练集与测试集
                X_train, y_train, X_test, y_test = X[train_idx], y[train_idx], X[test_idx], y[test_idx]
                print ('Fit fold', (j+1), '...')
                regr.fit(X_train, y_train)
                y_pred = regr.predict(X_test)                
                S_train[test_idx, i] = y_pred
                S_predict_i[:, j] = regr.predict(T)
            
            S_predict[:, i] = S_predict_i.mean(axis=1)

        nmse_score = cross_val_score(self.stacker, S_train, y, cv=5, scoring='neg_mean_squared_error')
        print('CV MSE:', -nmse_score)
        print('Stacker AVG MSE:', -nmse_score.mean(), 'Stacker AVG Score:', np.mean(np.divide(1, 1 + np.sqrt(-nmse_score))))

        self.stacker.fit(S_train, y)
        res = self.stacker.predict(S_predict)
        return res, S_train, S_predict

In [ ]:

# stacking_mode1 = Ridge(alpha=0.008, copy_X=True, fit_intercept=False, solver='auto', random_state=2)# stacki 
stacking_model = SVR(C=100, gamma=0.01, epsilon=0.01)
stacker = Stacker(5, stacking_model, regrs)
pred_stack, S_train_data, S_predict_data = stacker.fit_predict(all_X_train, all_y_train, sub_data)

In [ ]:
df_result['score'] = pred_stack

In [ ]:
index = df_result[df_result['ID'].isin(special_missing_ID)].index
df_result.loc[index, 'score'] = 0.379993053

In [ ]:
# c_index = df_result[df_result['ID'].isin(cleaned_sub_data_ID)].index
# df_result.loc[c_index, 'score'] = pred_clean_stack

In [ ]:
df_result.to_csv('../result/081502_08751.csv', index=False, header=False)